In [1]:
# Data Loading & first Pre-Processing steps
import numpy as np

#Read data

X = []
samples_x = []

f = open("genotypes.csv","r")

for i,line in enumerate(f):
    if(i == 0):
       continue
    
    sv = line.strip().split(",")
    samples_x.append(sv[0])
    
    row = []
    for element in sv[1:]:
        row.append(element)
    X.append(row)
    
f.close()

X = np.array(X)
samples_x = np.array(samples_x)

y = []
samples_y = []

f = open("phenotype_values.csv","r")

for i,line in enumerate(f):
    if(i == 0):
       continue
    sv = line.strip().split(",")
    samples_y.append(sv[0])
    y.append(float(sv[1]))
    
f.close()

y = np.array(y)
samples_y = np.array(samples_y)

#Replace "?" with np.nan
X = np.where(X == "?", np.nan, X)

# Match genotypes with phenotype values knn
truth_table = (samples_x[:,np.newaxis]==samples_y)
ind = np.where(truth_table==True)

samples_x = samples_x[ind[0]]
samples_y = samples_y[ind[1]]

X = X[ind[0],:]
y = y[ind[1]]

# Split into train data and test data

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=73)

In [2]:
#Encode nucleotide values

for j in range (X_train.shape[1]):
    
    values, counts = np.unique(X_train[:,j], return_counts=True)
    values=values[:2]
    counts=counts[:2]

    X_train[:,j] = np.where(X_train[:,j] == values[np.argmax(counts)], 0, X_train[:,j])
    X_train[:,j] = np.where(X_train[:,j] == values[np.argmin(counts)], 2, X_train[:,j])
    #Encode test data based on the known data
    X_test[:,j] = np.where(X_test[:,j] == values[np.argmax(counts)], 0, X_test[:,j]) 
    X_test[:,j] = np.where(X_test[:,j] == values[np.argmin(counts)], 2, X_test[:,j])
    
#Convert array from string to float
X_train = X_train.astype(np.float_)
X_test = X_test.astype(np.float_)

In [3]:
#Impute missing values on training data with KNNImputer
from sklearn.impute import KNNImputer

knn_imputer = KNNImputer (missing_values=np.nan, n_neighbors=3, weights="distance")
X_train = knn_imputer.fit_transform(X_train)

for i in range (X_test.shape[0]):
    x = X_test[i]
    x.shape = (1,5000)
    X_testr = knn_imputer.fit_transform(np.append(x,X_train,axis = 0))
    [test_data, R] = np.split(X_testr,[1],axis = 0)
    while i == 0:
        test = test_data
        break
    while i >= 1:
        test = np.append(test,test_data,axis = 0)
        break

for j in range (X_train.shape[1]):
    X_train[:,j] = np.where(X_train[:,j] < 1, 0.0, X_train[:,j])
    X_train[:,j] = np.where(X_train[:,j] >= 1, 2.0, X_train[:,j])
    test[:,j] = np.where(test[:,j] < 1, 0.0, test[:,j])
    test[:,j] = np.where(test[:,j] >= 1, 2.0, test[:,j])
    X_test = test

In [ ]:
#PCA was not always used please see the poster for replication

#standardise the data
from sklearn.preprocessing import StandardScaler 

scaler = StandardScaler()
Xn = scaler.fit_transform(X_train)

#compute the covariance matrix on the standardised data
n = Xn.shape[0]
C = 1/(n-1) * np.dot(Xn.T,Xn)

#eigendecompose the matrix  𝐂  into its eigenvalues and eigenvector
import numpy.linalg as linalg
d, V = linalg.eig(C)

#sort the eigenvalues in decreasing order, re-sort the columns of the eigenvector matrix 𝑉  using the indices from the sorting of the eigenvalues
ind = np.argsort(d)[::-1]
d = d[ind]
V = V[:,ind]

Xn1 = np.dot(Xn,V[:,0:750])
X_train = np.array(Xn1,dtype=float)

#calculate how much of the total variance the PCs account for
ratios_variance_explained = d/d.sum()
va = ratios_variance_explained[0:750].sum()

#transform test data
Xn3_Xtest = np.dot(test,V[:,0:750])
X_test = np.array(Xn3_Xtest,dtype=float)

In [ ]:
###MODELS###

In [ ]:
#KNN Regression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import KNNImputer
import numpy.linalg as linalg
import time
import numpy as np


K = np.arange(1,len(X_train)+1,1)
r2_trainknr = []
r2_testknr = []

start = time.process_time()
for k in K:
    knr = KNeighborsRegressor(n_neighbors=k)
    knr.fit(X_train,y_train)
    knr_predictions_training = knr.predict(X_train)
    knr_predictions_testing = knr.predict(X_test)
    r2_trainingknr = metrics.r2_score(y_train,knr_predictions_training)
    r2_testingknr = metrics.r2_score(y_test,knr_predictions_testing)
    r2_trainknr.append(r2_trainingknr)
    r2_testknr.append(r2_testingknr)

best_k = K[np.argmax(r2_testknr)]
rint("Best_k = " + str(best_k))
print("Best R2: %.4f" %np.max(r2_testknr))
print(r2_testknr)

#____________________________________________________________

delta = (time.process_time()-start)
print("Computation Time: %f s" %delta)

In [ ]:
#ElasticNetCV
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy.linalg as linalg
import sklearn.metrics as metrics
from sklearn.impute import KNNImputer
import time
import numpy as np

start = time.process_time()
l1r = np.arange(0.1,1.05,0.05)

#ElasticNetCV    
elncv = linear_model.ElasticNetCV(cv = 5, l1_ratio=l1r, max_iter=5000, random_state=73)
elncv.fit(X_train, y_train)
predictions_training = elncv.predict(X_train)
predictions_testing = elncv.predict(test)

print("MSE (Train):\t%.2f" % metrics.mean_squared_error(y_train, predictions_training))
print("R2 (Train):\t%.2f" % metrics.r2_score(y_train, predictions_training))
print("MSE (Test):\t%.2f" % metrics.mean_squared_error(y_test, predictions_testing))
print("R2 (Test):\t%.2f" % metrics.r2_score(y_test, predictions_testing))
print("The selected Alpha:\t%.f " %elncv.alpha_)
print("The selected L1-Ration:\t%.f " %elncv.l1_ratio_)

delta = time.process_time() - start
print("Computation Time: %f s" %delta)

In [5]:
#ElasticNet Model
from sklearn.linear_model import ElasticNet
import sklearn.metrics as metrics
import time

start = time.process_time()

eln =ElasticNet(alpha=0.05792901004687936, l1_ratio=0.25, max_iter=7000, random_state=73)
eln.fit(X_train, y_train)
predictions_training = eln.predict(X_train)
predictions_testing = eln.predict(X_test)

print("MSE (Train):\t%.2f" % metrics.mean_squared_error(y_train, predictions_training))
print("R2 (Train):\t%.2f" % metrics.r2_score(y_train, predictions_training))
print("MSE (Test):\t%.2f" % metrics.mean_squared_error(y_test, predictions_testing))
print("R2 (Test):\t%.2f" % metrics.r2_score(y_test, predictions_testing))

#____________________________________________________________

delta = (time.process_time()-start)
print("Computation Time: %f s" %delta)

MSE (Train):	0.65
R2 (Train):	0.94
MSE (Test):	3.41
R2 (Test):	0.66
Computation Time: 1209.324586 s
